In [1]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

Вопрос 1

Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины $X\sim N\left(\mu,\sigma^2\right)$ лежит в интервале $\mu\pm c \cdot \sigma$. Чему равно точное значение константы c? Округлите ответ до четырёх знаков после десятичной точки. 

In [2]:
round(scipy.stats.norm.ppf(1 - 0.003 / 2.), 4)

2.9677

Вопрос 5

В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [5]:
round(-104/11037.0 + 189/11034.0, 4)

0.0077

Вопрос 6

Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки. 

In [8]:
def proportions_confint_diff_ind(p1, p2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   

    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ 11034.0 + p2 * (1 - p2)/11037.0)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ 11034.0 + p2 * (1 - p2)/ 11037.0)
    
    return (left_boundary, right_boundary)

In [11]:
x = proportions_confint_diff_ind(189/11034.0, 104/11037.0)
round(x[1], 4)

0.0107

In [6]:
plus_asp = 104
neg_asp = 11037-104
plus_plac = 189
neg_plac = 11034-189
round(plus_plac/neg_plac*neg_asp/plus_asp,4)


1.8321

In [16]:
X1 = [1]*104 + [0]*(11037-104)
X1 = np.array(X1)
X2 = [1]*189 + [0]*(11034-189)
X2 = np.array(X2)

In [23]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples


def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

def chances(X):
    return len(X[X==1]) / len(X[X==0]) 

In [25]:
np.random.seed(0)
aspirin_chance = list(map(chances, get_bootstrap_samples(X1, 1000)))
placebo_chance = list(map(chances, get_bootstrap_samples(X2, 1000)))

In [28]:
print("95% confidence interval for the infarct odds with aspirin:",  stat_intervals(aspirin_chance, 0.05))
print("95% confidence interval for the infarct odds with placebo:",  stat_intervals(placebo_chance, 0.05))

95% confidence interval for the infarct odds with aspirin: [0.00757714 0.0113626 ]
95% confidence interval for the infarct odds with placebo: [0.01499402 0.01996672]


In [29]:
ratio = list(map(lambda x: x[1] / x[0], zip(aspirin_chance, placebo_chance)))

In [30]:
print("95% confidence interval for the ratio between odds",  stat_intervals(ratio, 0.05))

95% confidence interval for the ratio between odds [1.44419465 2.34321168]


In [31]:
round(stat_intervals(ratio, 0.05)[0], 4)

1.4442